# Homework week 6

## Question 3: Connecting to the kafka server

In [37]:
import json

from kafka import KafkaProducer
import pandas as pd

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

# because we run inside a docker container
server = 'host.docker.internal:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [39]:
cols_to_keep=['lpep_pickup_datetime',
'lpep_dropoff_datetime',
'PULocationID',
'DOLocationID',
'passenger_count',
'trip_distance',
'tip_amount']

trips_data = pd.read_csv("https://github.com/DataTalksClub/nyc-tlc-data/releases/download/green/green_tripdata_2019-10.csv.gz", usecols=cols_to_keep)

In [15]:
trips_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476386 entries, 0 to 476385
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   lpep_pickup_datetime   476386 non-null  datetime64[ns]
 1   lpep_dropoff_datetime  476386 non-null  datetime64[ns]
 2   PULocationID           476386 non-null  int64         
 3   DOLocationID           476386 non-null  int64         
 4   passenger_count        387007 non-null  float64       
 5   trip_distance          476386 non-null  float64       
 6   tip_amount             476386 non-null  float64       
dtypes: datetime64[ns](2), float64(3), int64(2)
memory usage: 25.4 MB


In [40]:
from time import time

t0 = time()



for i, row in trips_data.iterrows():
    producer.send('green-trips', value=dict(row))

producer.flush()

t1 = time()
took = t1 - t0